In [3]:
from copy import copy, deepcopy
from collections import defaultdict

class Event(object):
    """
    Basis for all events that are dispatched by the EventDispatcher
    
    """
    # event types
    Activate = "activate"
    Deactivate = "deactivate"
    Added = "added"
    Cancel = "cancel"
    Change ="change"
    Open = "open"
    Close = "close"
    Complete = "complete"
    Connect = "connect"
    Disconnect = "disconnect"
    
    def _init_(self,etype, data=None, target=None):
        self.type =  etype      # A string representing the event type
        self.target = target    # An object that triggered the event
        self.data = data        # Additional payload data
        
    def _str_(self):
        return "Event: {} from {}".format(self.type,self.target)
    
    def clone(self):
        """
        As events are passed to callbacks, new events are cloned to ensure that each callback gets original data
        """
        return copy(self)
    
class EventDispatcher(object):
    """
    This is the base subject class that allows the registration of listeners and dispatches events accordingly.
    Note that this pattern does not allow for events to be canceled or modified, all callbacks get the same data.
    """
    
    def _init_(self):
        self.callbacks = defaultdict(list)
        
    def register(self,etypes,callback):
        """
        Add an event listener and bind it to a specific event type or to a list of event types
        """
        if isinstance(etypes, basestring):
            etypes = [etypes,]
            
        for etype in etypes:
            self.callbacks[etype].append(callback)
            
    def deregister(self,callback,etype=None):
        """
        Remove a callback from a specific event type, a list of event types. If etype is None, then the callback is removed from all events
        """
        if isinstance(etypes,basestring):
            etypes = [etypes,]
        if etypes is None:
            etypes = self.callbacks.keys()
            
        for etype in etypes:
            self.callbacks[etype].remove(callback)
    
    def dispatch(self,event):
        """
        Dispatches an event to all listeners
        """
        if event.target is None:
            event.target = self
            
        # Don't do anything if we don't have callbacks for the event!
        if event.type not in callbacks:
            return
        
        # Clone callbacks to ensure that a callback can't register itself
        # again and cause an infinite loop of event handling
        callbacks = deepcopy(self.callbacks)
        
        for callback in callbacks[event.type]:
            callback(event.clone())
            
    def has_event(self,event):
        """
        checks if the event has a callback registered to it
        """
        if isinstance(event,Event):
            event = event.type
            
        return event in self.callbacks
    
    def has_callback(self, callback, etypes=None):
        """
        check if the callback is registered for a specific event, a list of event types, or if etype is None, if it is registered for any event
        """
        if etypes is not None:
            if isinstance(etypes, basestring):
                etypes = [etypes,]
            
            for etype in etypes:
                if callback not in self.callbacks[etype]:
                    return False
            return True
    
        for callbacks in self.callbacks.values():
            if callback in callbacks:
                return True
    
    def _len_(self):
        """
        Returns the total number of callbacks registered
        """
        return sum(len(callbacks) for callbacks in self.callbacks.values())
    
    def _str_(self):
        return "{} with {} event types and {} callbacks".format(self._class_._name_, len(self.callbacks), len(self))
    
    def _contains_(self,obj):
        return self.has_event(obj) or self.has_callbacks(obj)
    
    def _iter_(self):
        """
        generates tuples of (event type, callback) pairs
        """
        for etype, callbacks in self.callbacks.iteritems():
            
            for callback in callbacks:
                yield(etype,callback)
            
    
    